In [ ]:
import numpy as np
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

In [ ]:
books = pd.read_csv("books_with_categories.csv")
books.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age_of_book,description_word_count,simple_categories
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,21.0,199,Fiction
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,25.0,205,Fiction
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,0,43.0,14,Fiction
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,32.0,57,Fiction
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,23.0,45,Nonfiction


In [ ]:
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      device = 0,
                      top_k=None)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
books.isna().sum()

,0
isbn13,0
isbn10,0
title,0
subtitle,4226
authors,63
categories,33
thumbnail,214
description,0
published_year,0
average_rating,0


In [ ]:
books["description"][0 ]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [ ]:
emotional_labels = ["anger","disgust","fear","joy","neutral", "sadness","suprise",]
per_emotion_scores = {label:[] for label in emotional_labels}
predictions = classifier(books["description"][0].split("."))
for prediction in predictions:
  sorted_prediction = sorted(prediction,key = lambda x: x['label'])
  # print(sorted_prediction)
  for idx, label in enumerate(emotional_labels):
    # print(idx, label)
    per_emotion_scores[label].append(sorted_prediction[idx]["score"])

print({label: np.max(scores) for label, scores in per_emotion_scores.items()})

{'anger': np.float64(0.0641336739063263), 'disgust': np.float64(0.2735914885997772), 'fear': np.float64(0.928167998790741), 'joy': np.float64(0.9327971935272217), 'neutral': np.float64(0.6462162137031555), 'sadness': np.float64(0.9671575427055359), 'suprise': np.float64(0.7296026945114136)}


In [ ]:
emotional_labels = ["anger","disgust","fear","joy","neutral", "sadness","suprise"]
isbn = []
emotions_scores = {label: [] for label in emotional_labels}

In [ ]:
def calculate_max_emotion_scores(predictions):
  per_emotion_scores = {label:[] for label in emotional_labels}
  for prediction in predictions:
    sorted_prediction = sorted(prediction, key = lambda x: x['label'])
    for index, label in enumerate(emotional_labels):
      per_emotion_scores[label].append(sorted_prediction[index]["score"])
  return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [ ]:
for i in range(10):
  isbn.append(books["isbn13"][i])
  sentences = books["description"][i].split(".")
  predictions = classifier(sentences)
  max_scores =  calculate_max_emotion_scores(predictions)
  for label in emotional_labels:
    emotions_scores[label].append(max_scores[label])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
emotions_scores

{'anger': [np.float64(0.0641336739063263),
  np.float64(0.6126185059547424),
  np.float64(0.0641336739063263),
  np.float64(0.0641336739063263),
  np.float64(0.35148391127586365),
  np.float64(0.08141247183084488),
  np.float64(0.0641336739063263),
  np.float64(0.2322249710559845),
  np.float64(0.5381842255592346),
  np.float64(0.0641336739063263)],
 'disgust': [np.float64(0.2735914885997772),
  np.float64(0.3482844829559326),
  np.float64(0.10400670021772385),
  np.float64(0.10400670021772385),
  np.float64(0.15072263777256012),
  np.float64(0.1844954937696457),
  np.float64(0.10400670021772385),
  np.float64(0.727174699306488),
  np.float64(0.15585491061210632),
  np.float64(0.10400670021772385)],
 'fear': [np.float64(0.928167998790741),
  np.float64(0.9425278306007385),
  np.float64(0.05136282742023468),
  np.float64(0.9723208546638489),
  np.float64(0.36070623993873596),
  np.float64(0.09504325687885284),
  np.float64(0.05136282742023468),
  np.float64(0.05136282742023468),
  np.fl

In [ ]:
for i in tqdm(range(len(books))):
  isbn.append(books["isbn13"][i])
  sentences = books["description"][i].split(".")
  predictions = classifier(sentences)
  max_scores = calculate_max_emotion_scores(predictions)
  for label in emotional_labels:
    emotions_scores[label].append(max_scores[label])


100%|██████████| 6507/6507 [02:42<00:00, 39.94it/s]


In [ ]:
emotions_df = pd.DataFrame(emotions_scores)
emotions_df["isbn13"] = isbn

In [ ]:
emotions_df.head()

,anger,disgust,fear,joy,neutral,sadness,suprise,isbn13
0,0.064134,0.273591,0.928168,0.932797,0.646216,0.967158,0.729603,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.051363,0.565122,0.549477,0.111690,0.078766,9780006163831
3,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
4,0.351484,0.150723,0.360706,0.251881,0.732685,0.111690,0.078766,9780006280897


In [ ]:
books = pd.merge(books, emotions_df, on = "isbn13")
books.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,...,age_of_book,description_word_count,simple_categories,anger,disgust,fear,joy,neutral,sadness,suprise
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,...,21.0,199,Fiction,0.064134,0.273591,0.928168,0.932797,0.646216,0.967158,0.729603
1,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,...,21.0,199,Fiction,0.064134,0.273591,0.928168,0.932797,0.646216,0.967158,0.729603
2,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,...,25.0,205,Fiction,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545
3,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,...,25.0,205,Fiction,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545
4,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,...,43.0,14,Fiction,0.064134,0.104007,0.051363,0.565122,0.549477,0.111690,0.078766


In [ ]:
books.to_csv("books_with_emotions.csv", index = False)